In [2]:
import networkx as nx

# Step 1: Define the graph
G = nx.DiGraph()

# Example: Adding edges with capacity and delay
G.add_edge('A', 'B', capacity=10, delay=2)
G.add_edge('B', 'C', capacity=5, delay=1)
G.add_edge('A', 'C', capacity=15, delay=3)

# Step 2: Helper function to get path delay and check for contention
def get_path_and_delay(G, src, dest, packets, current_edge_usage):
    try:
        # Find the shortest path based on delay
        path = nx.shortest_path(G, source=src, target=dest, weight='delay')
    except nx.NetworkXNoPath:
        return None, float('inf')  # No path found
    
    path_delay = 0
    for u, v in zip(path[:-1], path[1:]):
        capacity = G[u][v]['capacity']
        delay = G[u][v]['delay']
        
        # Calculate waiting time if current usage exceeds capacity
        used_capacity = current_edge_usage.get((u, v), 0)
        if used_capacity + packets > capacity:
            return None, float('inf')  # Conflict if packets exceed available capacity
        
        path_delay += delay * (1 + used_capacity // capacity)
    
    return path, path_delay

# Step 3: Process requests in batches without contention
def process_requests(G, requests):
    total_time = 0
    remaining_requests = requests[:]
    current_edge_usage = {}

    while remaining_requests:
        batch_requests = []
        next_remaining_requests = []
        
        # Collect non-overlapping requests for the current batch
        for src, dest, packets in remaining_requests:
            path, path_delay = get_path_and_delay(G, src, dest, packets, current_edge_usage)
            
            if path:
                # Add the request to the batch if no contention
                batch_requests.append((src, dest, packets, path, path_delay))
                
                # Update edge usage for the batch
                for u, v in zip(path[:-1], path[1:]):
                    current_edge_usage[(u, v)] = current_edge_usage.get((u, v), 0) + packets
            else:
                # Postpone this request to the next round due to contention
                next_remaining_requests.append((src, dest, packets))
        
        # If batch is non-empty, process it and add its max delay to total time
        if batch_requests:
            max_batch_time = max(delay for _, _, _, _, delay in batch_requests)
            total_time += max_batch_time
        
        # Clear edge usage after each batch
        current_edge_usage.clear()
        remaining_requests = next_remaining_requests

    return total_time

# Example requests
requests = [
    ('A', 'C', 5),  # (source, destination, packets)
    ('A', 'B', 3),
    ('B', 'C', 2),
]

# Compute total time to serve all requests
time_required = process_requests(G, requests)
print(f"Total time required to serve all requests: {time_required}")


Total time required to serve all requests: 3
